In [22]:
#importing library
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Input File


In [25]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/Input.csv")[['URL_ID','URL']]

In [26]:
df=df.iloc[0:150]

In [27]:
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [28]:
df.drop('URL_ID',axis=1,inplace=True)

# Data extraction

In [29]:
df.head()

,URL
0,https://insights.blackcoffer.com/ai-in-healthc...
1,https://insights.blackcoffer.com/what-if-the-c...
2,https://insights.blackcoffer.com/what-jobs-wil...
3,https://insights.blackcoffer.com/will-machine-...
4,https://insights.blackcoffer.com/will-ai-repla...


In [30]:
#extracting text from all the url
url_id=1
for i in range(0,len(df)):
 
   j=df.iloc[i].values
   
   headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}#giving user access
   page=requests.get(j[0],headers=headers)#loading text in url
   soup=BeautifulSoup(page.content,'html.parser')#parsing url text
   content=soup.findAll(attrs={'class':'td-post-content'})#extracting only text part
   #print(content)
   
   if len(content) ==0:
         print('length is 0') 
         print('id', url_id) 
   else:
     content=content[0].text.replace('\xa0',"  ").replace('\n',"  ")#replace end line symbol with space 
     title=soup.findAll(attrs={'class':'entry-title'})#extracting title of website
     title=title[16].text.replace('\n',"  ").replace('/',"")
     text=title+ '.' +content#merging title and content text
     text=np.array(text)#converting to array form
     text.reshape(1,-1)#changing shape to 1d 
     df1=pd.Series(text)#creating series data frame
     b=str(url_id)+"."+'txt'#name of the text file
  # with open(b, 'a') as f:#creating text file 
   # df1.to_csv(f, line_terminator=',', index=False, header=False)
  # files.download(b)#downloading text file
   url_id+=1

length is 0
id 8
length is 0
id 21
length is 0
id 108


# Data analysis

In [31]:
#importing each extracted files
text=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/Input.csv",header=None)

In [32]:
#information of data frame
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       115 non-null    object
 1   1       115 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


In [33]:
#removing extra created column
text.drop(1,axis=1,inplace=True)

In [34]:
#converting type
text=text.astype(str)

In [35]:
#converting text to sentence
import re
a=text[0].str.split('([\.]\s)',expand=False)#splitting text on '.'
b=a.explode()#converting to rows
b=pd.DataFrame(b)#creating data frame
b.columns=['abc']


In [36]:
#removing . char from each rows
def abcd(x):    
    nopunc =[char for char in x if char != '.']
    return ''.join(nopunc)
b['abc']=b['abc'].apply(abcd)


In [37]:
#replacing emty space with null values
c=b.replace('',np.nan,regex=True)
c=c.mask(c==" ")
c=c.dropna()
c.reset_index(drop=True,inplace=True)

In [39]:
c

,abc
0,URL_ID
1,37
2,38
3,39
4,40
...,...
110,146
111,147
112,148
113,149


In [56]:
#importing nltk library and stopwords
import nltk
nltk.download('punkt')
import string


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [57]:
punc=[punc for punc in string.punctuation]

In [58]:
punc

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [59]:
#importing stop words files that are provided
StopWords_Auditor=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_Auditor.txt",header=None)
StopWords_Currencies=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_Currencies.txt",header=None,encoding="ISO-8859-1",sep='\n' )
StopWords_DatesandNumbers=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_DatesandNumbers.txt",header=None)
StopWords_Generic=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_Generic.txt",header=None)
StopWords_GenericLong=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_GenericLong.txt",header=None)
StopWords_Geographic=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_Geographic.txt",header=None)
StopWords_Names=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/StopWords/StopWords_Names.txt",header=None)


In [60]:
#creating func for removing stop words
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])


In [61]:
#applying func for each row
c['abc']=c['abc'].apply(text_process)

In [62]:
c

,abc
0,URL_ID
1,37
2,38
3,39
4,40
...,...
110,146
111,147
112,148
113,149


In [63]:
#importing master Dictionary
positive=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/MasterDictionary/positive-words.txt",header=None)
negative=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science Intern/MasterDictionary/negative-words.txt",header=None,encoding="ISO-8859-1",sep='\n' )


In [64]:
positive.columns=['abc']
negative.columns=['abc']
positive['abc']=positive['abc'].astype(str)
negative['abc']=negative['abc'].astype(str)

In [65]:
#positive and negative dictionary without stopwords
positive['abc']=positive['abc'].apply(text_process)
negative['abc']=negative['abc'].apply(text_process)

In [66]:
#positive list
length=positive.shape[0]
post=[]
for i in range(0,length):
   nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
   nopunc=''.join(nopunc)

   post.append(nopunc)

In [67]:
#negative list
length=negative.shape[0]
neg=[]
for i in range(0,length):
  nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
  nopunc=''.join(nopunc)
  neg.append(nopunc)

In [68]:
#importing tokenize library
from nltk.tokenize import word_tokenize

In [69]:
txt_list=[]
length=c.shape[0]
for i in range(0,length):
  txt=' '.join([word for word in c.iloc[i]])
  txt_list.append(txt)

In [74]:
#tokenization of text

for i in txt_list:
  tokenize_text += (word_tokenize(i))

In [75]:
print(tokenize_text)

['URL_ID', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150']


In [76]:
len(tokenize_text)

115

### 1) POSITIVE SCORE

In [77]:
positive_score=0
for i in tokenize_text:
  if(i.lower() in post):
    positive_score+=1
print('postive score=', positive_score)

postive score= 0


### 2) NEGATIVE SCORE

In [79]:
negative_score=0
for i in tokenize_text:
  if(i.lower() in neg):
    negative_score+=1
print('negative score=', negative_score)

negative score= 0


### 3) POLARITY SCORE

In [80]:
#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Polarity_Score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)
print('polarity_score=', Polarity_Score)

polarity_score= 0.0


### 4) SUBJECTIVITY SCORE

In [81]:
#Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
subjectiivity_score=(positive_score-negative_score)/((len(tokenize_text))+ 0.000001)
print('subjectivity_score',subjectiivity_score)

subjectivity_score 0.0


### 5) AVG SENTENCE LENGTH

In [82]:
length=c.shape[0]
avg_length=[]
for i in range(0,length):
  avg_length.append(len(c['abc'].iloc[i]))
avg_senetence_length=sum(avg_length)/len(avg_length)
print('avg sentence length=', avg_senetence_length)

avg sentence length= 2.4782608695652173


### 6) PERCENTAGE OF COMPLEX WORDS

In [83]:
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0


In [84]:
Percentage_of_Complex_words=complex_Word_Count/len(tokenize_text)
print('percentag of complex words= ',Percentage_of_Complex_words)

percentag of complex words=  0.0


### 7) FOG INDEX

In [85]:
#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
Fog_Index = 0.4 * (avg_senetence_length + Percentage_of_Complex_words)
print('fog index= ',Fog_Index )

fog index=  0.991304347826087


### 8) AVG NUMBER OF WORDS PER SENTENCE

In [86]:
length=c.shape[0]
avg_length=[]
for i in range(0,length):
  a=[word.split( ) for word in c.iloc[i]]
  avg_length.append(len(a[0]))
  a=0
#avg
avg_no_of_words_per_sentence=sum(avg_length)/length
print("avg no of words per sentence= ",avg_no_of_words_per_sentence)

avg no of words per sentence=  1.0


### 9) COMPLEX WORD COUNT

In [87]:
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
  if(count>2):
   complex_Word_Count+=1
  count=0
print('complex words count=',  complex_Word_Count)

complex words count= 0


### 10) WORD COUNT

In [88]:
word_count=len(tokenize_text)
print('word count= ', word_count)

word count=  115


### 11) SYLLABLE PER WORD

In [89]:
vowels=['a','e','i','o','u']
import re
count=0
for i in tokenize_text:
  x=re.compile('[es|ed]$')
  if x.match(i.lower()):
   count+=0
  else:
    for j in i:
      if(j.lower() in vowels ):
        count+=1
syllable_count=count
print('syllable_per_word= ',syllable_count)

syllable_per_word=  2


### 12) PERSONAL PRONOUNS

In [90]:
pronouns=['i','we','my','ours','us' ]
import re
count=0
for i in tokenize_text:
  if i.lower() in pronouns:
   count+=1
personal_pronouns=count
print('personal pronouns= ',personal_pronouns )

personal pronouns=  0


### 13) AVG WORD LENGTH

In [91]:
count=0
for i in tokenize_text:
  for j in i:
    count+=1
avg_word_length=count/len(tokenize_text)
print('avg word= ', avg_word_length)

avg word=  2.4782608695652173


In [92]:
data={'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':Polarity_Score,'subjectiivity_score':subjectiivity_score,'avg_senetence_length':avg_senetence_length,'Percentage_of_Complex_words':Percentage_of_Complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':avg_no_of_words_per_sentence,'complex_Word_Count':complex_Word_Count,'word_count':word_count,'syllable_count':syllable_count,'personal_pronouns':personal_pronouns,'avg_word_length':avg_word_length}

In [94]:
output=pd.DataFrame()
output=output.append(data,ignore_index=True)
output.columns=['positive_score','negative_score','Polarity_Score','subjectiivity_score','avg_senetence_length','Percentage_of_Complex_words','Fog_Index','avg_no_of_words_per_sentence','complex_Word_Count','word_count','syllable_count','personal_pronouns','avg_word_length']
output

,positive_score,negative_score,Polarity_Score,subjectiivity_score,avg_senetence_length,Percentage_of_Complex_words,Fog_Index,avg_no_of_words_per_sentence,complex_Word_Count,word_count,syllable_count,personal_pronouns,avg_word_length
0,0.0,0.0,0.0,0.0,2.478261,0.0,0.991304,1.0,0.0,115.0,2.0,0.0,2.478261


In [96]:
with open('output.csv', 'a') as f:#creating text file 
  output.to_csv(f, index=False, header=False)
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>